In [36]:
from langchain.document_loaders import TextLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.chains import VectorDBQA
from langchain.chains import RetrievalQA
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [37]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain.schema import SystemMessage

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a chatbot having a conversation with a human."
        ),  # The persistent system prompt
        MessagesPlaceholder(
            variable_name="chat_history"
        ),  # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),  # Where the human input will injected
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

llm = ChatOpenAI(model='gpt-4')

# llm = OpenAI(model='gpt-4-vision-preview')

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
)

# chat_llm_chain.predict(human_input="my name is angad")

In [38]:
# from langchain.schema.messages import HumanMessage, SystemMessage

# chat = ChatOpenAI(model="gpt-4-vision-preview")
# chat.invoke(
#     [
#         HumanMessage(
#             content=[
#                 {"type": "text", "text": """I'm giving you an image of a floorplan.
#                  """},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://www.grundriss-schmiede.de/images/buerogrundriss/buerogrundriss.png",
#                         "detail": "auto",
#                     },
#                 },
#             ]
#         )
#     ]
# )

In [39]:
def split_string(input_string, max_length=180):
    # max_length = 125
    chunks = []

    current_chunk = ""
    for char in input_string:
        if char == '\n' or len(current_chunk) == max_length:
            chunks.append(current_chunk)
            current_chunk = ""
        else:
            current_chunk += char

    # Append the last chunk
    if current_chunk:
        chunks.append(current_chunk)

    return chunks

In [40]:
q_string = """Imagine an x-y cartesian plane. Point A is at (100,100) and Point B is at (500,200).
What is the distance from point A to point B.
There are the following 3 rules to be obeyed during the journey.
Rule 1: IF x = 200 THEN 200<y<300 for any point during the journey.
Rule 2: IF x = 400 THEN 0<y<100 for any point during the journey.
Rule 3: There is an outside bounding box so 0.5<x<599.5 & 0.5<y<299.5 for any point during the journey.

PLEASE ENSURE THAT NO RULE SHOULD BE VIOLATED. I REPEAT. NO RULE."""
# Keeping these rules in mind, tell if the distance to both point A and point B is more than 45 from any point
# in the cartesian plane""" 

# 3. There is an obstruction going from (200,0) to (200,200). So i can't go through this obstruction.

# q_string = """Give code to convert an image of a floorplan to a maze to go from one staircase
# to another in python."""

q_string = """Given a floorplan with walls and two staircases, I want to convert it into an X-Y plane representation and generate a maze. 
The walls in the floorplan should be translated into barriers in the maze, and the staircases should serve as entry and exit points.
 Provide me with a detailed description or visualization of the resulting maze in the X-Y plane format.
"""

large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)

As an AI text-based model, I don't have the ability to directly create visualizations. However, I can provide a detailed step-by-step process on how you can convert your floor plan
into a maze in the X-Y plane format:

1. **Coordinate System**: Begin by defining your X-Y plane. The X-axis can represent the width of the floorplan, while the Y-axis can represent the height. Each point (x, y) on thi
 plane corresponds to a specific point on your floor plan.

2. **Walls as Barriers**: Translate the walls in your floor plan into barriers in your maze. Depending on the resolution of your grid, each wall might occupy one or multiple cells.
You can represent barriers in a variety of ways. If you're coding, you could use a two-dimensional array, where each cell has a value of 0 (for open space) or 1 (for a barrier).

3. **Staircases as Entry and Exit Points**: Choose one staircase to be the entry point and the other to be the exit point of your maze. Their positions can be defined by their X an

In [41]:
q_string = """I have a room which is 600 metres long and 300 metres wide. it has a wall that runs from (200,0) 
to (200,200) and another than runs from (400,100) to (400,300). One staircase is at (100,100) and another at (500,500). Conver this into the maze like i told before and solve.
Consider each unit on the grid as a 1m by 1m"""

large_string = chat_llm_chain.predict(human_input=q_string)

# Split the string using the newline character
lines = split_string(large_string)

# Print each line
for line in lines:
    print(line)


As a text-based AI, I can't visualize this for you, but I can guide you on how to create this maze and solve it.

1. **Coordinate System**: Set up a grid of 600 by 300. Each unit cell corresponds to 1 metre.

2. **Walls as Barriers**: The wall that runs from (200,0) to (200,200) on your plane is a barrier. It spans across 200 units vertically on the grid and is located at x=200.

3. **Staircases as Entry and Exit Points**: You have one staircase at (100,100) which will serve as the entry point (S), and another one at (500,500) which will serve as the exit p
int (E).

4. **Generating the Maze**: Since the only wall is a straight line and doesn't split the room, the simplest path from the entry to the exit would be a straight line avoiding the wa
l. One could navigate by following the left wall (or right wall, depending on preference) until they reach the exit.

5. **Solving the Maze**: Start at point S (100,100). Move right (increase x direction) until you hit the wall at (200,100). The